In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

/Users/matsalexander/.pyenv/versions/3.10.13/envs/gluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
class Master:
    @staticmethod
    def timestamp(df, name_time_col):
        # Create a sample DataFrame
        df_stamped = df.copy()

# Convert the name_time_col to datetime format
        df_stamped[name_time_col] = pd.to_datetime(df_stamped[name_time_col])

# Calculate 'days_since_jan' and 'hour' and create new columns for them
        df_stamped['days_since_jan'] = df_stamped[name_time_col].dt.dayofyear
        df_stamped['hour'] = df_stamped[name_time_col].dt.hour

# Drop the original datetime column
        df_stamped.drop(name_time_col, axis=1, inplace=True)

# Display the DataFrame
        return df_stamped

    @staticmethod
    def averaged(df, name_time_col):
        # df.groupby(pd.Grouper(key=time_col, freq="1H")) du burde bruke denne. Resample fyller nan rows for alle timer som ikke er i datasettet.
        df_averaged = df.copy()
        df_averaged = pd.DataFrame(df_averaged)
        df_averaged[name_time_col] = pd.to_datetime(df_averaged[name_time_col])
        df_averaged.set_index(name_time_col, inplace=True)
        df_resampled = df_averaged.resample('1H').mean()
        
        # all_nan_mask = df_resampled.isnull().all(axis=1)
        # df_resampled = df_resampled[~all_nan_mask]
        
        df_resampled.reset_index(inplace=True)

        return df_resampled

    def averaged_redundant(df, name_time_col, column_to_sum):
        df_averaged = df.copy()
        df_averaged[name_time_col] = pd.to_datetime(df_averaged[name_time_col])
        df_averaged.set_index(name_time_col, inplace=True)

        # Resample and take the mean for the entire DataFrame
        df_resampled = df_averaged.resample('1H').mean()

        # Resample the specific column you want to sum over 15-minute intervals
        sum_resampled = df_averaged[column_to_sum].resample('15T').sum()

        # Adjust the 1-hour resampled DataFrame with the summed values for every 15 minutes
        df_resampled[column_to_sum] = sum_resampled.resample('1H').sum()

        df_resampled.reset_index(inplace=True)

        return df_resampled

    def averaged_test(df, name_time_col):

        df_averaged = df.copy()
        df_averaged[name_time_col] = pd.to_datetime(df_averaged[name_time_col])

    # Extract date and hour from the timestamp
        df_averaged['date'] = df_averaged[name_time_col].dt.date
        df_averaged['hour'] = df_averaged[name_time_col].dt.hour

    # Group by date and hour to get the mean
        df_resampled = df_averaged.groupby(
            ['date', 'hour']).mean().reset_index()

    # Reconstruct the timestamp from the date and hour
        df_resampled[name_time_col] = pd.to_datetime(df_resampled['date'].astype(
            str) + ' ' + df_resampled['hour'].astype(str) + ':00:00')
        df_resampled.drop(columns=['date', 'hour'], inplace=True)

        return df_resampled

    @staticmethod
    def target_join(df1, df2, df_target):
        df_merge = pd.concat([df1.copy(), df2.copy()], ignore_index=True)
        df_merge = df_merge[df_merge['date_forecast'].isin(
            df_target.copy()['time'])]
        return df_merge

    @staticmethod
    def randomForest(df_features, df_targets):

        X_train, X_test, Y_train, Y_test = train_test_split(
            df_features, df_targets, test_size=0.2, random_state=42)

        # If you meant to keep only the first column of Y_train and Y_test
        Y_train = Y_train.iloc[:, 0]
        Y_test = Y_test.iloc[:, 0]

        # Handle NaN values in feature matrices
        X_train.fillna(0, inplace=True)
        X_test.fillna(0, inplace=True)

        # Handle NaN values in target variables with caution, or maybe consider dropping NaNs if applicable.
        # If you decide to fill NaN in targets:
        Y_train.fillna(0, inplace=True)
        Y_test.fillna(0, inplace=True)

        # Create a Random Forest Regressor model
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, Y_train)  # Train the model with the training data

        # Make predictions on the test data
        predictions = model.predict(X_test)
        # Calculate the Mean Absolute Error
        mae = mean_absolute_error(Y_test, predictions)
        # print(f"Mean Absolute Error: {mae}")
        return mae

    def inner_join_old(feature_df, target_df):
        # Inner join on 'date_forecast' and 'time' columns

        target_df = target_df.dropna(subset=['pv_measurement'])

        merged_df = feature_df.merge(
            target_df, left_on='date_forecast', right_on='time', how='inner')

    # Extract columns from merged_df for each DataFrame
        cols_feature = feature_df.columns
        cols_target = target_df.columns

        aligned_feature_df = merged_df[cols_feature]
        aligned_target_df = merged_df[cols_target]

    # Drop all rows from the original DataFrames
        feature_df.drop(feature_df.index, inplace=True)
        target_df.drop(target_df.index, inplace=True)

        # Append the aligned rows using pandas.concat
        feature_df = pd.concat(
            [feature_df, aligned_feature_df], ignore_index=True)
        target_df = pd.concat(
            [target_df, aligned_target_df], ignore_index=True)

        return feature_df, target_df

    def inner_join(feature_df, target_df):
        # Assuming 'pv_measurement' is the target variable column in target_df.
        # Perform an inner join on 'date_forecast' from feature_df and 'time' from target_df.
        # This will only include rows where the keys match in both DataFrames.

        target_df = target_df.dropna(subset=['pv_measurement'])

        merged_df = feature_df.merge(target_df[['time', 'pv_measurement']],
                                     left_on='date_forecast',
                                     right_on='time',
                                     how='inner')

        # If you do not want to keep the 'time' column from target_df after the join,
        # you can drop it from the resulting DataFrame.
        merged_df.drop('time', axis=1, inplace=True)

        return merged_df

    def onehot_estimated(df, estimated_bool):

        if estimated_bool:
            df["estimated"] = 1
            df["observed"] = 0

        else:
            df["estimated"] = 0
            df["observed"] = 1

        return df

    def onehot_location(df, location):

        # This is garbage, write cleaner

        if location == "A":
            df["A"] = 1
            df["B"] = 0
            df["C"] = 0

        if location == "B":
            df["A"] = 0
            df["B"] = 1
            df["C"] = 0

        if location == "C":
            df["A"] = 0
            df["B"] = 0
            df["C"] = 1

        return df


In [4]:
target_A = pd.read_parquet("../A/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
#test_A=pd.read_parquet("../A/X_test_estimated.parquet")
A_observed=pd.read_parquet("../A/X_train_observed.parquet")
A_estimated=pd.read_parquet("../A/X_train_estimated.parquet")
target_B = pd.read_parquet("../B/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
B_observed = pd.read_parquet("../B/X_train_observed.parquet")
B_estimated=pd.read_parquet("../B/X_train_estimated.parquet")
target_C = pd.read_parquet("../C/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
C_observed=pd.read_parquet("../C/X_train_observed.parquet")
C_estimated=pd.read_parquet("../C/X_train_estimated.parquet")

# test sets
test_A=pd.read_parquet("../A/X_test_estimated.parquet")
test_B=pd.read_parquet("../B/X_test_estimated.parquet")
test_C=pd.read_parquet("../C/X_test_estimated.parquet")
test_A




,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-04-30 07:00:04,2023-05-01 00:00:00,4.4,1.286,912.700012,0.0,0.0,1041.199951,0.0,271.700012,0.0,0.0,0.0,0.0,80.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.900024,0.0,0.0,1000.700012,1007.000000,0.0,0.0,80.300003,1013.299988,NaN,0.0,0.0,0.0,0.0,10.520000,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0
1,2023-04-30 07:00:04,2023-05-01 00:15:00,4.3,1.287,912.700012,0.0,0.0,1054.800049,0.0,271.700012,0.0,0.0,0.0,0.0,77.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.700012,0.0,0.0,1000.599976,1006.900024,0.0,0.0,80.300003,1013.200012,NaN,0.0,0.0,0.0,0.0,14.203000,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0
2,2023-04-30 07:00:04,2023-05-01 00:30:00,4.3,1.287,912.700012,0.0,0.0,1068.300049,0.0,271.600006,0.0,0.0,0.0,0.0,73.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.599976,0.0,0.0,1000.500000,1006.700012,0.0,0.0,80.300003,1013.000000,NaN,0.0,0.0,0.0,0.0,17.868999,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0
3,2023-04-30 07:00:04,2023-05-01 00:45:00,4.3,1.287,912.700012,0.0,0.0,1081.900024,0.0,271.600006,0.0,0.0,0.0,0.0,69.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.500000,0.0,0.0,1000.400024,1006.599976,0.0,0.0,80.199997,1012.900024,NaN,0.0,0.0,0.0,0.0,21.514000,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0
4,2023-04-30 07:00:04,2023-05-01 01:00:00,4.3,1.287,NaN,0.0,0.0,1095.400024,0.0,271.600006,0.0,0.0,0.0,0.0,64.500000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.299988,0.0,0.0,1000.200012,1006.500000,0.0,0.0,80.199997,1012.799988,NaN,0.0,0.0,0.0,0.0,25.135000,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2023-07-02 07:00:31,2023-07-03 22:45:00,9.1,1.207,2090.600098,0.0,0.0,1629.400024,0.0,282.700012,0.0,0.0,0.0,0.0,99.300003,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.299988,0.0,0.0,980.799988,986.700012,0.0,0.0,82.199997,992.599976,NaN,0.0,0.0,-0.0,0.0,351.855988,-3.282,0.1,285.399994,99.400002,39931.601562,2.1,1.7,-1.3,0.0
2876,2023-07-02 07:00:31,2023-07-03 23:00:00,9.1,1.207,2015.800049,0.0,0.0,1537.300049,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.400024,0.0,0.0,980.900024,986.799988,0.0,0.0,82.500000,992.700012,NaN,0.0,0.0,-0.0,0.0,355.317993,-3.469,0.1,285.299988,100.000000,39574.300781,2.0,1.6,-1.3,0.0
2877,2023-07-02 07:00:31,2023-07-03 23:15:00,9.1,1.208,2015.699951,0.0,0.0,1471.400024,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.500000,0.0,0.0,981.000000,986.900024,0.0,0.0,82.800003,992.799988,NaN,0.0,0.0,-0.0,0.0,358.785004,-3.556,0.1,285.299988,100.000000,39627.199219,2.0,1.6,-1.2,0.0
2878,2023-07-02 07:00:31,2023-07-03 23:30:00,9.0,1.208,2015.599976,0.0,0.0,1405.500000,0.0,282.700012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,993.599976,0.0,0.0,981.000000,986.900024,0.0,0.0,83.199997,992.900024,NaN,0.0,0.0,-0.0,0.0,2.253000,-3.541,0.1,285.200012,100.000000,39691.800781,2.0,1.6,-1.1,0.0


In [5]:
time_col = "date_forecast"
print("test_A.shape)",test_A.shape)
df = test_A.copy()
df[time_col] = pd.to_datetime(df['date_forecast'])

df = df.groupby(pd.Grouper(key=time_col, freq="1H")).mean()
all_nan_mask = df.isnull().all(axis=1)
df = df[~all_nan_mask]
df = df.reset_index()



print("number of days" , df[time_col].dt.date.nunique())

print(df[time_col].dt.date.unique())
print(df.shape)
df.head(30)


test_A.shape) (2880, 47)
number of days 30
[datetime.date(2023, 5, 1) datetime.date(2023, 5, 6)
 datetime.date(2023, 5, 14) datetime.date(2023, 5, 18)
 datetime.date(2023, 5, 20) datetime.date(2023, 5, 21)
 datetime.date(2023, 5, 22) datetime.date(2023, 5, 23)
 datetime.date(2023, 5, 25) datetime.date(2023, 5, 27)
 datetime.date(2023, 5, 28) datetime.date(2023, 5, 30)
 datetime.date(2023, 6, 1) datetime.date(2023, 6, 2)
 datetime.date(2023, 6, 3) datetime.date(2023, 6, 4)
 datetime.date(2023, 6, 6) datetime.date(2023, 6, 9)
 datetime.date(2023, 6, 11) datetime.date(2023, 6, 13)
 datetime.date(2023, 6, 14) datetime.date(2023, 6, 18)
 datetime.date(2023, 6, 19) datetime.date(2023, 6, 22)
 datetime.date(2023, 6, 26) datetime.date(2023, 6, 28)
 datetime.date(2023, 6, 29) datetime.date(2023, 6, 30)
 datetime.date(2023, 7, 2) datetime.date(2023, 7, 3)]
(720, 46)


/var/folders/b0/slnc__2x31jczblnsfgfmjw00000gn/T/ipykernel_59418/4034204498.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(pd.Grouper(key=time_col, freq="1H")).mean()


,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-05-01 00:00:00,4.325,1.28675,912.700012,0.000000e+00,0.000000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000e+00,74.949997,6.0,0.000,0.000,0.000,0.000,0.000,0.00,1.00,1013.674988,0.0,0.0,1000.549988,1006.799988,0.0,0.0,80.275002,1013.099976,NaN,0.0,0.0,0.0,0.000,16.026501,-10.541000,0.000,273.799988,74.949997,29907.500000,3.950,2.100,3.350,0.0
1,2023-05-01 01:00:00,4.275,1.28600,1482.099976,0.000000e+00,0.000000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000e+00,77.474998,6.0,0.000,0.000,0.000,0.000,0.000,0.00,1.00,1013.150024,0.0,0.0,1000.049988,1006.299988,0.0,0.0,79.824997,1012.599976,NaN,0.0,0.0,0.0,0.000,30.497250,-7.894500,0.000,273.799988,77.474998,29519.074219,3.825,1.925,3.300,0.0
2,2023-05-01 02:00:00,4.150,1.28375,1791.300049,0.000000e+00,0.000000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000e+00,88.099998,6.0,0.000,0.000,0.000,0.000,0.000,0.00,1.00,1012.674988,0.0,0.0,999.500000,1005.799988,0.0,0.0,78.000000,1012.049988,NaN,0.0,0.0,0.0,0.000,44.517250,-3.815500,0.000,273.849976,88.099998,31009.125000,3.650,1.750,3.200,0.0
3,2023-05-01 03:00:00,4.025,1.28200,2312.875000,1.012442e+04,11.675000,1179.849976,0.0,270.649994,9.375000,16845.226562,2.100000,3.765350e+03,68.599998,6.0,0.000,0.000,0.000,0.000,0.000,0.75,0.50,1012.174988,0.0,0.0,998.974976,1005.224976,0.0,0.0,75.625000,1011.525024,NaN,0.0,0.0,0.0,0.000,58.083000,1.412500,0.000,273.899994,68.599998,34552.500000,3.500,1.450,3.150,0.0
4,2023-05-01 04:00:00,3.900,1.28100,2198.299805,1.417486e+05,76.875000,920.049988,0.0,270.375000,47.400002,102209.703125,25.450001,4.957120e+04,66.300003,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1011.724976,0.0,0.0,998.549988,1004.750000,0.0,0.0,74.224998,1011.049988,NaN,0.0,0.0,0.0,0.000,71.341003,7.468500,0.000,273.924988,66.300003,35483.875000,3.325,1.300,3.050,0.0
5,2023-05-01 05:00:00,4.000,1.27600,1957.625000,4.716072e+05,190.574997,1419.300049,0.0,270.549988,86.425003,240972.250000,88.225006,2.046413e+05,74.074997,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1011.349976,0.0,0.0,998.125000,1004.349976,0.0,0.0,71.099998,1010.599976,NaN,0.0,0.0,0.0,0.000,84.557755,14.017750,0.000,274.250000,74.074997,35425.449219,3.200,1.400,2.875,0.0
6,2023-05-01 06:00:00,4.000,1.26825,1903.300049,9.212486e+05,322.149994,1299.400024,0.0,270.824982,111.074997,355522.968750,181.500000,4.855580e+05,58.550003,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1011.000000,0.0,0.0,997.750000,1003.950012,0.0,0.0,66.599998,1010.174988,NaN,0.0,0.0,0.0,0.000,98.091248,20.711750,0.000,274.950012,58.550003,33709.773438,3.550,2.275,2.675,0.0
7,2023-05-01 07:00:00,4.000,1.26250,NaN,1.394139e+06,450.575012,743.299988,0.0,270.899994,130.100006,434118.187500,266.750000,8.068865e+05,48.549999,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1010.724976,0.0,0.0,997.450012,1003.625000,0.0,0.0,63.375000,1009.849976,NaN,0.0,0.0,0.0,0.000,112.359497,27.171749,0.000,275.575012,48.549999,26871.000000,4.200,3.475,2.350,0.0
8,2023-05-01 08:00:00,4.100,1.25975,1381.199829,1.830129e+06,562.450012,805.375000,0.0,271.024994,159.225006,520811.437500,315.200012,1.047560e+06,61.075001,6.0,0.000,0.000,0.000,0.000,0.000,1.00,0.00,1010.450012

In [6]:
def add_time_since_calucation(df):
        df["date_calc"] = pd.to_datetime(df["date_calc"])
        df["calculated_ago"] = (
            df["date_forecast"] - df["date_calc"]).dt.total_seconds()
        df["calculated_ago"] = df["calculated_ago"].fillna(
            0)
        return df

A_observed = Master.averaged(A_observed,"date_forecast")
A_observed = Master.onehot_estimated(A_observed,False)
A_estimated = Master.averaged(A_estimated,"date_forecast")
A_estimated = Master.onehot_estimated(A_estimated,True)

B_observed = Master.averaged(B_observed,"date_forecast")
B_observed = Master.onehot_estimated(B_observed,False)
B_estimated = Master.averaged(B_estimated,"date_forecast")
B_estimated = Master.onehot_estimated(B_estimated,True)


C_observed = Master.averaged(C_observed,"date_forecast")
C_observed = Master.onehot_estimated(C_observed,False)
C_estimated = Master.averaged(C_estimated,"date_forecast")
C_estimated = Master.onehot_estimated(C_estimated,True)



#Do so for test data as well
test_A=Master.averaged_test(test_A,"date_forecast")
test_A=Master.onehot_estimated(test_A,True)
test_A=Master.onehot_location(test_A,"A")

test_B=Master.averaged_test(test_B,"date_forecast")
test_B=Master.onehot_estimated(test_B,True)
test_B=Master.onehot_location(test_B,"B")



test_C=Master.averaged_test(test_C,"date_forecast")
test_C=Master.onehot_estimated(test_C,True)
test_C=Master.onehot_location(test_C,"C")


#C_estimated.head()
print(test_A.shape)
test_A.head()


(720, 51)


/var/folders/b0/slnc__2x31jczblnsfgfmjw00000gn/T/ipykernel_59418/2036744127.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_resampled = df_averaged.resample('1H').mean()
/var/folders/b0/slnc__2x31jczblnsfgfmjw00000gn/T/ipykernel_59418/2036744127.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_resampled = df_averaged.resample('1H').mean()
/var/folders/b0/slnc__2x31jczblnsfgfmjw00000gn/T/ipykernel_59418/2036744127.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only o

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,date_forecast,estimated,observed,A,B,C
0,4.325,1.28675,912.700012,0.000000,0.000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000,74.949997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.674988,0.0,0.0,1000.549988,1006.799988,0.0,0.0,80.275002,1013.099976,NaN,0.0,0.0,0.0,0.0,16.026501,-10.5410,0.0,273.799988,74.949997,29907.500000,3.950,2.100,3.35,0.0,2023-05-01 00:00:00,1,0,1,0,0
1,4.275,1.28600,1482.099976,0.000000,0.000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000,77.474998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.150024,0.0,0.0,1000.049988,1006.299988,0.0,0.0,79.824997,1012.599976,NaN,0.0,0.0,0.0,0.0,30.497250,-7.8945,0.0,273.799988,77.474998,29519.074219,3.825,1.925,3.30,0.0,2023-05-01 01:00:00,1,0,1,0,0
2,4.150,1.28375,1791.300049,0.000000,0.000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000,88.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1012.674988,0.0,0.0,999.500000,1005.799988,0.0,0.0,78.000000,1012.049988,NaN,0.0,0.0,0.0,0.0,44.517250,-3.8155,0.0,273.849976,88.099998,31009.125000,3.650,1.750,3.20,0.0,2023-05-01 02:00:00,1,0,1,0,0
3,4.025,1.28200,2312.875000,10124.424805,11.675,1179.849976,0.0,270.649994,9.375000,16845.226562,2.100000,3765.350098,68.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.75,0.5,1012.174988,0.0,0.0,998.974976,1005.224976,0.0,0.0,75.625000,1011.525024,NaN,0.0,0.0,0.0,0.0,58.083000,1.4125,0.0,273.899994,68.599998,34552.500000,3.500,1.450,3.15,0.0,2023-05-01 03:00:00,1,0,1,0,0
4,3.900,1.28100,2198.299805,141748.593750,76.875,920.049988,0.0,270.375000,47.400002,102209.703125,25.450001,49571.199219,66.300003,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1011.724976,0.0,0.0,998.549988,1004.750000,0.0,0.0,74.224998,1011.049988,NaN,0.0,0.0,0.0,0.0,71.341003,7.4685,0.0,273.924988,66.300003,35483.875000,3.325,1.300,3.05,0.0,2023-05-01 04:00:00,1,0,1,0,0


In [7]:
A_observed_estimated  = pd.concat([A_observed,A_estimated],ignore_index=True)

A_observed_estimated  = Master.onehot_location(A_observed_estimated,"A")

B_observed_estimated  = pd.concat([B_observed,B_estimated],ignore_index=True)
B_observed_estimated = Master.onehot_location(B_observed_estimated,"B")

C_observed_estimated  = pd.concat([C_observed,C_estimated],ignore_index=True)
C_observed_estimated = Master.onehot_location(C_observed_estimated,"C")

#A_observed_estimated=Master.inner_join(A_estimated,target_A)
A_observed_estimated_target= Master.inner_join(A_observed_estimated,target_A)
B_observed_estimated_target = Master.inner_join(B_observed_estimated,target_B)
C_observed_estimated_target = Master.inner_join(C_observed_estimated,target_C)

Dt64 feature --> Two features: days_since_jan, hour

In [8]:
A_observed_estimated=Master.timestamp(A_observed_estimated,"date_forecast")
B_observed_estimated=Master.timestamp(B_observed_estimated,"date_forecast")
C_observed_estimated=Master.timestamp(C_observed_estimated,"date_forecast")
test_A=Master.timestamp(test_A,"date_forecast")
test_B=Master.timestamp(test_B,"date_forecast")
test_C=Master.timestamp(test_C,"date_forecast")
A_B_C_all=pd.concat([A_observed_estimated_target,B_observed_estimated_target,C_observed_estimated_target])

Concatinating A,B,C with all accessible values, this is jalla jalla

In [9]:
test_A_B_C = pd.concat([test_A,test_B,test_C],ignore_index=True)
A_B_C_all = Master.timestamp(A_B_C_all,"date_forecast")


In [10]:
A_B_C_all.describe()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,estimated,observed,A,B,C,pv_measurement,days_since_jan,hour
count,92951.000000,92951.000000,76534.000000,9.295100e+04,92951.000000,86213.000000,92951.000000,92951.000000,92951.000000,9.295100e+04,92951.000000,9.295100e+04,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,4213.0,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,92951.000000,93023.000000,93023.000000,93023.000000,93023.000000,93023.000000,93023.000000,93023.000000,93023.000000
mean,6.017393,1.255435,2888.300537,5.151552e+05,143.098038,1735.995117,0.006799,275.237976,39.395203,1.418185e+05,50.245178,1.808859e+05,67.086441,11.401738,0.117096,0.009727,0.231608,0.029244,0.058533,0.483303,0.564284,1009.502563,0.005657,0.084348,995.818848,1001.949585,0.747828,0.009566,73.670593,1008.107666,250.0,0.193164,0.000022,0.000273,0.090299,179.648560,-1.206875,0.056897,279.430634,73.692551,33025.011719,3.038167,0.664565,0.685095,-0.000008,0.118218,0.881782,0.366415,0.353063,0.280522,287.022737,176.121196,11.501338
std,2.711861,0.036567,2536.682617,8.185716e+05,227.959961,1809.297363,0.234531,6.829573,60.518574,2.152898e+05,112.917160,4.012631e+05,34.269562,7.877236,0.779764,0.104943,1.223007,0.275208,0.477040,0.485974,0.483166,13.085625,0.029169,0.330071,13.004988,13.063836,5.298545,0.041121,14.229107,13.124816,0.0,1.253925,0.004181,0.004249,0.237841,97.282532,23.970707,0.105794,6.515625,34.021942,17913.982422,1.760291,2.802007,1.878808,0.005994,0.322868,0.322868,0.481827,0.477925,0.449257,766.411327,109.535913,6.920154
min,0.500000,1.139250,27.799999,0.000000e+00,0.000000,27.500000,-1.000000,247.425003,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,944.375000,0.000000,0.000000,929.974976,935.750000,0.000000,0.000000,19.575001,941.549988,250.0,0.000000,0.000000,0.000000,0.000000,6.983000,-49.931999,0.000000,258.024994,0.000000,132.375000,0.025000,-7.225000,-8.400000,-0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,1.000000,0.000000
25%,4.025000,1.230250,1087.606201,0.000000e+00,0.000000,591.924988,0.000000,270.750000,0.000000,0.000000e+00,0.000000,0.000000e+00,42.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1001.400024,0.000000,0.000000,987.775024,993.849976,0.000000,0.000000,64.199997,999.974976,250.0,0.000000,0.000000,0.000000,0.000000,94.678749,-18.599751,0.000000,274.899994,53.224998,16862.800781,1.675000,-1.350000,-0.575000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,78.000000,6.000000
50%,5.450000,1.255000,1887.887451,1.047617e+04,1.650000,1164.525024,0.000000,274.975006,0.925000,1.051303e+04,0.000000,0.000000e+00,79.949997,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,1.000000,1010.349976,0.000000,0.000000,996.750000,1002.849976,0.000000,0.000000,76.000000,1009.000000,250.0,0.000000,0.000000,0.000000,0.000000,179.979752,-0.864500,0.

In [11]:
test_A_B_C.describe()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,estimated,observed,A,B,C,days_since_jan,hour
count,2160.000000,2160.000000,1590.000000,2.160000e+03,2160.000000,1937.000000,2160.000000,2160.000000,2160.000000,2.160000e+03,2160.000000,2.160000e+03,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,0.0,2160.0,2160.0,2160.0,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000,2160.0,2160.0,2160.000000,2160.000000,2160.000000,2160.000000,2160.000000
mean,8.202084,1.232774,3065.770508,1.227747e+06,341.040741,1875.397461,0.036111,280.778992,84.755867,3.051201e+05,114.341972,4.116306e+05,64.090988,12.333333,0.000509,0.000312,0.000509,0.000509,0.000509,0.793519,0.241204,1016.780457,0.007663,0.065972,1002.946045,1008.982666,0.016435,0.013067,70.843506,1015.044434,NaN,0.0,0.0,0.0,0.060637,179.180542,20.293180,0.064525,284.749359,69.259583,33348.164062,2.945822,1.652465,-0.182477,0.000359,1.0,0.0,0.333333,0.333333,0.333333,156.433333,11.500000
std,2.191513,0.032043,2951.748047,1.101274e+06,306.855469,2093.077637,0.182849,4.364160,78.432159,2.777555e+05,171.784576,6.107051e+05,37.460999,8.261587,0.014000,0.008819,0.014000,0.014000,0.014000,0.385461,0.413672,9.725667,0.031816,0.235240,9.640645,9.738119,0.468430,0.051688,15.604269,9.837888,NaN,0.0,0.0,0.0,0.199754,103.925461,18.630058,0.109135,5.830846,37.764587,15336.857422,1.714616,2.569777,1.490788,0.005502,0.0,0.0,0.471514,0.471514,0.471514,16.975835,6.923789
min,3.200000,1.142750,46.450001,0.000000e+00,0.000000,30.225000,-1.000000,268.075012,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,986.025024,0.000000,0.000000,971.724976,977.575012,0.000000,0.000000,24.599998,983.400024,NaN,0.0,0.0,0.0,0.000000,13.439500,-11.573000,0.000000,273.700012,0.000000,1532.175049,0.100000,-4.300000,-4.300000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,121.000000,0.000000
25%,6.650000,1.209437,949.168701,4.874178e+04,17.806250,505.424988,0.000000,277.924988,7.975000,3.417926e+04,0.000000,4.922562e+02,30.131249,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1011.493744,0.000000,0.000000,997.775024,1003.750000,0.000000,0.000000,60.275002,1009.743744,NaN,0.0,0.0,0.0,0.000000,89.979689,2.425875,0.000000,279.849976,36.625001,19856.712891,1.550000,-0.200000,-1.300000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,143.000000,5.750000
50%,8.050000,1.238000,1674.250000,9.333753e+05,299.512512,1037.525024,0.000000,281.049988,75.224998,2.649308e+05,13.475000,6.548056e+04,77.500000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1020.387482,0.000000,0.000000,1006.199982,1012.250000,0.000000,0.000000,73.587502,1018.337494,NaN,0.0,0.0,0.0,0.000000,180.959869,19.797500,0.000000,284.799988,92.237499,37066.824219,2.750000,1.600000,-0.250000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,154.500000,11.500000
75%,10.000000,1.259500,4054.912598,2.276978e+06,662.018738,2426.399902,0.000000,284.375000,134.418751,4.880685e+05,187.493748,6

In [12]:
test_A.describe()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,estimated,observed,A,B,C,days_since_jan,hour
count,720.000000,720.000000,532.000000,7.200000e+02,720.000000,650.000000,720.000000,720.000000,720.000000,7.200000e+02,720.000000,7.200000e+02,720.000000,720.0,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.0,720.000000,720.000000,720.000000,0.0,720.0,720.0,720.0,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.0,720.0,720.0,720.0,720.0,720.0,720.000000,720.000000
mean,8.205903,1.233150,3096.332764,1.227651e+06,341.014008,1891.609253,0.034722,280.800171,84.902321,3.056483e+05,114.781693,4.132130e+05,64.132294,6.0,0.000347,0.000174,0.000347,0.000347,0.000347,0.793750,0.225347,1016.797241,0.007476,0.063889,1003.811279,1009.852600,0.0,0.013333,70.750069,1015.920654,NaN,0.0,0.0,0.0,0.061146,179.181381,20.294441,0.063924,284.723358,69.309792,33599.617188,3.075139,1.739271,-0.218993,0.0,1.0,0.0,1.0,0.0,0.0,156.433333,11.500000
std,2.178425,0.032103,2945.732422,1.101479e+06,306.911926,2128.155273,0.174453,4.339592,78.590202,2.783206e+05,172.185043,6.121724e+05,37.369354,0.0,0.006583,0.004658,0.006583,0.006583,0.006583,0.385538,0.405328,9.731263,0.031335,0.230456,9.620932,9.717662,0.0,0.052354,15.531467,9.817863,NaN,0.0,0.0,0.0,0.201748,103.971336,18.630127,0.108792,5.811303,37.675526,15615.333008,1.795147,2.714267,1.488172,0.0,0.0,0.0,0.0,0.0,0.0,16.983703,6.926999
min,3.200000,1.151000,53.299999,0.000000e+00,0.000000,30.225000,0.000000,268.100006,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,986.025024,0.000000,0.000000,973.799988,979.625000,0.0,0.000000,24.824999,985.450012,NaN,0.0,0.0,0.0,0.000000,13.441751,-11.534750,0.000000,273.799988,0.000000,2104.725098,0.100000,-4.275000,-4.300000,0.0,1.0,0.0,1.0,0.0,0.0,121.000000,0.000000
25%,6.693750,1.209000,953.000000,4.879987e+04,17.950000,509.556274,0.000000,278.024994,8.087500,3.418273e+04,0.025000,7.144188e+02,32.675001,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1011.549988,0.000000,0.000000,998.418747,1004.437515,0.0,0.000000,60.368752,1010.487503,NaN,0.0,0.0,0.0,0.000000,89.981750,2.447438,0.000000,279.849976,38.918751,19721.299805,1.593750,-0.300000,-1.300000,0.0,1.0,0.0,1.0,0.0,0.0,143.000000,5.750000
50%,8.050000,1.238625,1712.699951,9.337900e+05,299.575012,1035.875000,0.000000,281.049988,75.299999,2.656778e+05,13.950000,6.590085e+04,76.875000,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1020.437500,0.000000,0.000000,1007.475006,1013.562500,0.0,0.000000,73.550003,1019.662506,NaN,0.0,0.0,0.0,0.000000,180.960999,19.800626,0.000000,284.774994,91.625000,36758.650391,2.950000,1.737500,-0.300000,0.0,1.0,0.0,1.0,0.0,0.0,154.500000,11.500000
75%,9.981250,1.260000,4097.362305,2.276145e+06,661.737488,2437.175049,0.000000,284.381248,134.231251,4.891316e+05,184.731247,6.564714e+05,99.949997,6.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1023.849976,0.000000,0.000000,1010.924988,1017.000000,0.0,0.000000,83.275002,1023.031265,NaN,0.0,0.0,0.0,0.000000,271.244804,38.861813,0.10000

In [13]:
# compare datasets
%autoreload
from pipeline_henni import Pipeline
pipe = Pipeline()

ModuleNotFoundError: No module named 'pipeline_henni'

In [ ]:
df1 = pipe.get_data("A")
df2 = pipe.get_data("B")
df3 = pipe.get_data("C")
df4 = pipe.get_combined_data()

hf1 = A_observed_estimated_target
hf2 = B_observed_estimated_target
hf3 = C_observed_estimated_target
hf4 = A_B_C_all
# hf4[""]
#rename days_since_jan to day_of_year
hf4.rename(columns={"days_since_jan": "day_of_year"}, inplace=True)

In [ ]:
different_columns = set(df4.columns).symmetric_difference(A_B_C_all.columns)
different_columns


set()

In [ ]:
hf4 = hf4[df4.columns].reset_index(drop=True)

In [ ]:
are_columns_equal = df1.columns.equals(df2.columns)
is_data_equal = df1.equals(df2)
print(are_columns_equal)
print(is_data_equal)

True
False


In [ ]:
# compare length of datasets
hfs = [hf1,hf2,hf3,hf4]
dfs = [df1,df2,df3,df4]
for hf,df in zip(hfs,dfs):
    print(hf.shape,df.shape)
    different_columns = set(df.columns).symmetric_difference(hf.columns)
    what = df.columns.difference(hf.columns)
    print(different_columns)
    print(what)
lenght1,lenght2 = len(hf4), len(df4)
print(lenght1,lenght2)

# hvis du legger til alle verdiene 

(34085, 52) (34085, 50)
{'day_of_year', 'date_forecast', 'B', 'A', 'hour', 'C'}
Index(['day_of_year', 'hour'], dtype='object')
(32843, 52) (32843, 50)
{'day_of_year', 'date_forecast', 'B', 'A', 'hour', 'C'}
Index(['day_of_year', 'hour'], dtype='object')
(26095, 52) (26095, 50)
{'day_of_year', 'date_forecast', 'B', 'A', 'hour', 'C'}
Index(['day_of_year', 'hour'], dtype='object')
(93023, 53) (93023, 53)
set()
Index([], dtype='object')
93023 93023


In [ ]:
hf4 = hf[df4.columns]
def absolute_value_for_wind_columns( df: pd.DataFrame):
    wind_columns = [col for col in df.columns]
    df[wind_columns] = df[wind_columns].abs()
    return df
hf4 = absolute_value_for_wind_columns(hf4)

def make_all_values_float(df: pd.DataFrame):
    df = df.astype(float)
    return df

df4 = make_all_values_float(df4)
hf4 = make_all_values_float(hf4)
df4 = df4.replace(-0.0, 0.0)
hf4 = hf4.replace(-0.0, 0.0)


# differences
sub2 = df4[["sun_elevation:d", "wind_speed_u_10m:ms", "wind_speed_v_10m:ms"]]
# sub2
df4.columns

Index(['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K',
       'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx',
       'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms', 'wind_speed_v_10

In [ ]:
# result = df4.compare(hf4)
sub1 = hf4[["sun_elevation:d", "wind_speed_u_10m:ms", "wind_speed_v_10m:ms"]]
sub1

,sun_elevation:d,wind_speed_u_10m:ms,wind_speed_v_10m:ms
0,3.77425,3.575,0.500
1,4.35725,3.350,0.275
2,3.30950,2.950,0.750
3,0.82250,2.600,0.875
4,3.05125,2.350,0.925
...,...,...,...
93018,0.20150,3.600,1.875
93019,5.20400,2.950,2.125
93020,8.98450,2.625,2.400
93021,11.27050,2.325,2.325


In [ ]:
equal = df4.equals(hf4)
print("Are the dataframes equal?", equal)


Are the dataframes equal? False


In [ ]:
equal_columns = df4.columns.equals(hf4.columns)
equal_data = df4.eq(hf4).all().all()
print("Do the dataframes have the same columns?", equal_columns)
print("Do the dataframes have the same values?", equal_data)


Do the dataframes have the same columns? True
Do the dataframes have the same values? False


In [ ]:
differences = df4 - hf4

# show me all rows that have values other than 0
# get all columns where there are values other than 0
non_zero_columns = df4.columns[(df4 > 0.1).any()]
differences = differences[non_zero_columns]
differences
column_sums = df4.sum()
column_sums

# greater = differences[(differences != 0) & (~differences.isna())].any(axis=1)
# show_me = differences[greater]
# show_me


absolute_humidity_2m:gm3          5.593227e+05
air_density_2m:kgm3               1.166940e+05
ceiling_height_agl:m              2.210532e+08
clear_sky_energy_1h:J             4.788419e+10
clear_sky_rad:W                   1.330110e+07
cloud_base_agl:m                  1.496654e+08
dew_or_rime:idx                   5.480500e+03
dew_point_2m:K                    2.558364e+07
diffuse_rad:W                     3.661823e+06
diffuse_rad_1h:J                  1.318217e+10
direct_rad:W                      4.670340e+06
direct_rad_1h:J                   1.681353e+10
effective_cloud_cover:p           6.235752e+06
elevation:m                       1.059803e+06
fresh_snow_12h:cm                 1.088420e+04
fresh_snow_1h:cm                  9.041250e+02
fresh_snow_24h:cm                 2.152817e+04
fresh_snow_3h:cm                  2.718250e+03
fresh_snow_6h:cm                  5.440700e+03
is_day:idx                        4.492350e+04
is_in_shadow:idx                  5.245075e+04
msl_pressure:

In [ ]:
are_rows_equal = all(df4.iloc[i].equals(hf1.iloc[i]) for i in range(len(df1)))
are_rows_equal



False